In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model, load_model, Model
from tensorflow.keras import optimizers, activations
from tensorflow.keras.layers import Dense, Flatten, LSTM, Bidirectional, SimpleRNN,Dropout
from tensorflow.keras.losses import logcosh
from tensorflow.metrics import mean_relative_error
from tensorflow.keras.layers import LeakyReLU
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt
import os
import csv
from tensorflow.keras import regularizers
from matplotlib.pyplot import MultipleLocator
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

In [2]:
def extract_time(line):
    data = line.replace("->","").lstrip().split("  ")[-1].split(" ")
#     print(data)
    start_cost = data[0].split("..")[0].replace("(cost=","")
#     print(start_cost)
    end_cost = data[0].split("..")[1]
    rows = data[1].replace("rows=","")
    width = data[2].replace("width=","").replace(")","")
    a_start_cost = data[4].split("..")[0].replace("time=","")
    a_end_cost = data[4].split("..")[1]
    a_rows = data[5].replace("rows=","") 
    return float(start_cost),float(end_cost),float(rows),float(width),float(a_start_cost),float(a_end_cost),float(a_rows)

In [3]:
# file_name_column_min_max_vals = "/home/slm/cardinality/learnedcardinalities/data/column_min_max_vals.csv"
# with open(file_name_column_min_max_vals, 'r') as f:
#     data_raw = list(list(rec) for rec in csv.reader(f, delimiter=','))
#     column_min_max_vals = {}
#     for i, row in enumerate(data_raw):
#         if i == 0:
#             continue
#         column_min_max_vals[row[0]] = [float(row[1]), float(row[2])]

In [4]:
def normalize_data(val,column_name,column_min_max_vals):
    min_val = column_min_max_vals[column_name][0]
    max_val = column_min_max_vals[column_name][1]
    val = float(val)
    if(val>max_val):
        val = max_val
    elif(val<min_val):
        val = min_val
    val = float(val)
    val_norm = (val - min_val) / (max_val - min_val)
    return val_norm


In [5]:
def is_not_number(s):
    try:
        float(s)
        return False
    except ValueError:
        pass
    try:
        import unicodedata
        unicodedata.numeric(s)
        return False
    except (TypeError, ValueError):
        pass
    return True

In [6]:
def get_data_and_label(path):
    plans = sorted(os.listdir(path))
    sentences = []
    rows = []
    pg = []
    d = {}
    for file in sorted(plans):
        with open(path+'/'+file,'r') as f:
#             print(file)
            plan = f.readlines()
            for i in range(len(plan)-2):
                if("Seq Scan" in plan[i]):
                    _start_cost,_end_cost,_rows,_width,_a_start_cost_,_a_end_cost,_a_rows = extract_time(plan[i])
                    if(len(plan[i].strip().split("  "))==2):
                        _sentence = " ".join(plan[i].strip().split("  ")[0].split(" ")[:-1]) + " "
                        table = plan[i].strip().split("  ")[0].split(" ")[4]
                    else:
                        _sentence = " ".join(plan[i].strip().split("  ")[1].split(" ")[:-1]) + " "
                        table = plan[i].strip().split("  ")[1].split(" ")[4]
                    if("actual" not in plan[i+1] and "Plan" not in plan[i+1]):
                        _sentence += plan[i+1].strip()
#                         print(_sentence)
                    else:
                        _sentence += table
                        _sentence += _sentence
#                         _sentence = _sentence + ' full'
#                         break
    #                     pass
#                         print(_sentence)
                    _sentence = _sentence.replace(": "," ").replace("(","").replace(")","").replace("'","").replace("::bpchar","")\
                        .replace("[]","").replace(","," ").replace("\\","").replace("::numeric","").replace("  "," ")\
                        .replace("Seq Scan on ","").strip()
#                     print(_sentence)
                    sentence = []
                    ll = _sentence.split(" ")
                    for cnt in range(len(ll)):                 
                        if is_not_number(ll[cnt]):
                            sentence.append(ll[cnt])
                        else:
                            try:
                                sentence.append(normalize_data(ll[cnt],table+'.'+str(ll[cnt-2]),column_min_max_vals))
                            except:
    #                             print(sentence)
                                pass
#                     print(sentence)
#                     if(tuple(sentence) not in d):
#                         d[tuple(sentence)] = 0
                    sentences.append(tuple(sentence))
                    rows.append(_a_rows)
                    pg.append(_rows)
    return sentences,rows,pg


In [7]:
path = r'/home/jitao/hierarchical_attention/data/deep_cardinality'
sentences,rows,pg = get_data_and_label(path)

In [8]:
vocabulary = []
for sentence in sentences:
    for word in sentence:
        if(word not in vocabulary and is_not_number(word)):
#             print(word)
            vocabulary.append(word)
print(len(vocabulary))
vocab_size = len(vocabulary)
print(vocabulary)

31
['movie_info_idx', 'Filter', 'info_type_id', '>', 'title', 'kind_id', '=', 'movie_keyword', 'keyword_id', 'cast_info', 'person_id', 'AND', 'role_id', 'mkmovie_keyword', 'mk', 'ttitle', 't', '<', 'production_year', 'movie_info', 'mimovie_info', 'mi', 'movie_companies', 'mcmovie_companies', 'mc', 'cicast_info', 'ci', 'mi_idxmovie_info_idx', 'mi_idx', 'company_id', 'company_type_id']


31
['movie_info_idx', 'Filter', 'info_type_id', '>', 'title', 'kind_id', '=', 'production_year', 'movie_keyword', 'keyword_id', 'cast_info', 'person_id', 'AND', 'role_id', 'mkmovie_keyword', 'mk', 'ttitle', 't', '<', 'movie_info', 'mimovie_info', 'mi', 'movie_companies', 'mcmovie_companies', 'mc', 'cicast_info', 'ci', 'company_id', 'company_type_id', 'mi_idxmovie_info_idx', 'mi_idx']

In [9]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
_vocabulary = np.array(vocabulary)
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(_vocabulary)
encoded = to_categorical(integer_encoded)
vocab_dict = {}
for v,e in zip(vocabulary,encoded):
    vocab_dict[v] = np.reshape(np.array(e),(1,vocab_size))

In [10]:
def prepare_data_and_label(sentences,rows):
    data = []
    label = []
    for sentence,row in zip(sentences,rows):
        _s = []
        for word in sentence:
            if(is_not_number(word)):
                _tmp = np.column_stack((np.array([0]),vocab_dict[word]))
                _tmp = np.reshape(_tmp,(vocab_size+1))
                assert(len(_tmp)==vocab_size+1)
                _s.append(_tmp)
            else:
#                 print(word)
#                 _tmp = np.full((vocab_size+1),word)
                _tmp = np.column_stack((np.array([float(word)]),np.zeros((1,vocab_size))))
                _tmp = np.reshape(_tmp,(vocab_size+1))
#                 print(_tmp)
                assert(len(_tmp)==vocab_size+1)
                _s.append(_tmp)
        data.append(np.array(_s))
        label.append(row)
    return data,label

In [11]:
data,label = prepare_data_and_label(sentences,rows)

In [12]:
def normalize_labels(labels, min_val=None, max_val=None):
# log tranformation withour normalize
    labels = np.array([np.log(float(l)) for l in labels]).astype(np.float32)
    return labels,0,1

In [13]:
label_norm, min_val, max_val = normalize_labels(label)

In [14]:
max_len = 0
for sentence in sentences:
    if(len(sentence) > max_len):
        max_len = len(sentence)
print(max_len)
padded_sentences = pad_sequences(data, maxlen=max_len, padding='pre',dtype='float32')

7


In [15]:
print(np.shape(padded_sentences))
print(np.shape(label_norm))

(176483, 7, 32)
(176483,)


In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(padded_sentences, label_norm, test_size=0.8, random_state=40)
pg_train,pg_test,_y_train,_y_test = train_test_split(pg,label_norm,test_size=0.2,random_state=40)
print(np.shape(X_train),np.shape(X_test))
print(np.shape(y_train),np.shape(y_test))

(35296, 7, 32) (141187, 7, 32)
(35296,) (141187,)


In [17]:
model = Sequential()
# model.add(Embedding(vocab_size, 5, input_length=max_len))
# model.add(SimpleRNN(64, return_sequences=True,activation='relu',input_shape=(max_len,vocab_size+1)))
# model.add(SimpleRNN(64,return_sequences=True,activation='relu'))
model.add(Bidirectional(SimpleRNN(128, return_sequences=True,activation='relu'),input_shape=(max_len,vocab_size+1)))
model.add(Bidirectional(SimpleRNN(128,return_sequences=True,activation='relu')))
# model.add(Bidirectional(SimpleRNN(64,return_sequences=True,activation='relu')))
# model.add(Bidirectional(SimpleRNN(64,return_sequences=True,activation='relu')))

model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(128,activation='relu'))
model.add(Dense(64,activation='relu'))#,kernel_regularizer=regularizers.l2(0.05)))
# model.add(Dense(64,activation='relu'))
model.add(Dense(64,activation='relu'))#,kernel_regularizer=regularizers.l2(0.05)))
# model.add(Dense(32,activation='relu'))
model.add(Dense(1, activation='relu'))

# model.compile(optimizer=optimizers.Adagrad(lr=0.01), loss=custom_loss(min_val,max_val), metrics=['mse','mae'])
model.compile(optimizer=optimizers.Adagrad(lr=0.01,decay=0.0001), loss='mse', metrics=['mse','mae'])

model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (None, 7, 256)            41216     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 7, 256)            98560     
_________________________________________________________________
flatten (Flatten)            (None, 1792)              0         
______________________________________________

In [18]:
# model.fit(X_train,y_train,validation_split=0.2,epochs=100, batch_size=8,shuffle=True)
model.fit(padded_sentences,label_norm,validation_split=0.2,epochs=140, batch_size=128,shuffle=True)

Train on 141186 samples, validate on 35297 samples
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Epoch 1/140
141186/141186 [==============================] - 37s 264us/sample - loss: 5.7400 - mean_squared_error: 5.7400 - mean_absolute_error: 1.1310 - val_loss: 1.5508 - val_mean_squared_error: 1.5508 - val_mean_absolute_error: 0.8134
Epoch 2/140
141186/141186 [==============================] - 41s 290us/sample - loss: 1.6618 - mean_squared_error: 1.6618 - mean_absolute_error: 0.8492 - val_loss: 1.5169 - val_mean_squared_error: 1.5169 - val_mean_absolute_error: 0.7492
Epoch 3/140
141186/141186 [==============================] - 38s 270us/sample - loss: 1.6032 - mean_squared_error: 1.6032 - mean_absolute_error: 0.8205 - val_loss: 2.8890 - val_mean_squared_error: 2.8890 - val_mean_absolute_error: 1.2577
Epoch 4/140
141186/141186 [==============================] - 41s 288us/sample - loss: 1.5824 - mean_squared_error: 1.

In [19]:
# model.save('./final/embedding_model_bak.h5') 
# model = tf.keras.models.load_model('./final/embedding_model_bak.h5')

# validation

In [20]:
# test_path = r'/data1/slm/datasets/JOB/synthetic/'
# # test_path = r'/data/slm/datasets/JOB/job-light/'
# # test_path = r'/data1/slm/datasets/JOB/cardinality/'
# test_sentences,test_rows,test_pg = get_data_and_label(test_path)
# test_data,test_label = prepare_data_and_label(test_sentences,test_rows)
# print(np.shape(test_data),np.shape(test_label))

In [21]:
# # label_norm, min_val, max_val = normalize_labels(label)
# max_len = 0
# for test_sentence in test_sentences:
#     if(len(test_sentence) > max_len):
#         max_len = len(test_sentence)
# print(max_len)
# test_padded_sentences = pad_sequences(test_data, maxlen=max_len, padding='pre',dtype='float32')

In [22]:
# print(np.shape(test_padded_sentences))

In [23]:
# pred = model.predict(test_padded_sentences)

In [24]:
# _pg = plt.scatter(test_label,test_pg,c='b')
# my = plt.scatter(test_label,np.exp(pred),c='r')
# plt.legend([_pg, my], ['pg', 'my'], loc='lower right', scatterpoints=1)
# _max = max(test_label)
# plt.plot([0,_max],[0,_max],c="y")
# plt.plot([0,_max],[0,2*_max],c="y")
# plt.plot([0,_max],[0,0.5*_max],c="y")
# plt.savefig("./results/test_embedding_result.png")

In [25]:
# import math
# result = np.exp(pred.flatten())/test_label
# pg_result = np.array(test_pg)/np.array(test_label)
# ax = plt.subplot(1, 2, 1)
# ax.boxplot(np.log(np.array(result)),showfliers=False)
# # ax.set_yticks(np.arange(0, 10))
# # ax.set_yticklabels(math.e**np.arange(-10, 10))
# ax.set_title('my q-error')
# ax2 = plt.subplot(1,2,2)
# ax2.boxplot(np.log(np.array(pg_result)),showfliers=False)
# # ax2.set_yticklabels(math.e**np.arange(-10, 10))
# ax2.set_title('pg q-error')
# # plt.savefig("test_box-2.png")
# plt.show()

In [26]:
# def print_qerror(preds_norm, labels_unnorm):
#     qerror = []
#     preds_unnorm = np.exp(preds_norm)-1
# #     labels_unnorm = np.exp(labels_norm)-1
#     for i in range(len(preds_unnorm)):
#         if preds_unnorm[i] > float(labels_unnorm[i]):
#             qerror.append(preds_unnorm[i] / float(labels_unnorm[i]))
#         else:
#             qerror.append(float(labels_unnorm[i]) / float(preds_unnorm[i]))

#     print("Median: {}".format(np.median(qerror)))
#     print("90th percentile: {}".format(np.percentile(qerror, 90)))
#     print("95th percentile: {}".format(np.percentile(qerror, 95)))
#     print("99th percentile: {}".format(np.percentile(qerror, 99)))
#     print("Max: {}".format(np.max(qerror)))
#     print("Mean: {}".format(np.mean(qerror)))

In [27]:
# print_qerror(pred,test_label)

bak better

In [28]:
intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.layers[6].output)
# intermediate_output = intermediate_layer_model.predict(test_padded_sentences)

intermediate_output = intermediate_layer_model.predict(padded_sentences)

In [29]:
# scan_label = []
# scan_label.append(['table','detail'])
# for sentence in sentences:
#     tmp = []
# #     print(sentence)
#     table = sentence[0]
#     tmp.append(table)
#     if(len(sentence)>2):
#         tmp.append(' '.join(str(each) for each in sentence[2:]))
#     else:
#         tmp.append(table)
#     scan_label.append(tmp)
# #     break


In [30]:
# np.savetxt("vectors.csv", intermediate_output, delimiter="\t")
# np.savetxt("labels.csv",scan_label,fmt='%s',delimiter = "\t")

In [31]:
print(np.shape(intermediate_output))
# print(np.shape(scan_label))

(176483, 64)


In [32]:
np.save("model_parameter/featuer_deep_cardinality.npy",intermediate_output)

In [33]:
intermediate_output[:3]
len(intermediate_output)

176483